# Import libraries

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv

# Import 2003 and 2004 data

In [3]:
df_aqi2003 = pd.read_csv("/Users/jadeg/Documents/UW/Spring/Erdos/annual_aqi_by_county_2003.csv")  
df_aqi2004 = pd.read_csv("/Users/jadeg/Documents/UW/Spring/Erdos/annual_aqi_by_county_2004.csv")

In [8]:
print(len(df_aqi2003))
print(len(df_aqi2004))

1136
1121


# Remove unwanted columns from both data

In [11]:
exclude_cols = ['Year'] 
exclude_set = set(exclude_cols)

df_aqi2003_filt = df_aqi2003.drop(columns=[col for col in df_aqi2003.columns if col in exclude_set])
df_aqi2004_filt = df_aqi2004.drop(columns=[col for col in df_aqi2004.columns if col in exclude_set])

# Define key columns
key_cols = ['State', 'County']

# Merge to keep only common rows by State and County
df_aqi2003_filt = df_aqi2003_filt.merge(df_aqi2004_filt [key_cols], on=key_cols, how='inner')
df_aqi2004_filt = df_aqi2004_filt .merge(df_aqi2003_filt[key_cols], on=key_cols, how='inner')

In [12]:
print(len(df_aqi2003_filt))
print(len(df_aqi2004_filt))

1102
1102


# Average Leftover data

In [19]:
numeric_cols = [col for col in df_aqi2003_filt.columns if col not in key_cols]

df1_numeric = df_aqi2003_filt[numeric_cols].apply(pd.to_numeric, errors='coerce')
df2_numeric = df_aqi2004_filt[numeric_cols].apply(pd.to_numeric, errors='coerce')

df_avg_end = (df1_numeric + df2_numeric) / 2

In [21]:
df_avg_end
df_avg = pd.concat([df_aqi2003_filt[key_cols], df_avg_end], axis=1)
print(df_avg)

        State      County  Days with AQI  Good Days  Moderate Days  \
0     Alabama     Baldwin          274.5      162.0           99.0   
1     Alabama     Barbour           32.0        9.0           22.5   
2     Alabama        Clay          278.5      161.0          110.5   
3     Alabama     Colbert          296.5      187.5          105.5   
4     Alabama      DeKalb          361.0      218.0          133.0   
...       ...         ...            ...        ...            ...   
1097  Wyoming      Platte           58.0       55.0            3.0   
1098  Wyoming    Sheridan          237.5      172.0           63.0   
1099  Wyoming    Sublette          335.5      255.0           79.0   
1100  Wyoming  Sweetwater          365.5      304.0           55.0   
1101  Wyoming       Teton          342.5      276.0           66.0   

      Unhealthy for Sensitive Groups Days  Unhealthy Days  \
0                                    13.0             0.5   
1                                    

# Replace state name with abbreviation

In [22]:
# US state name to abbreviation dictionary
state_to_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR',
    'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE',
    'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
    'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM',
    'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND',
    'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA',
    'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
    'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV',
    'Wisconsin': 'WI', 'Wyoming': 'WY',
}

# Replace state names in the "State" column with their abbreviations
df_avg['State'] = df_avg['State'].map(state_to_abbrev)

# save data as .npz

In [23]:
np.savez_compressed('pp_airquality.npz', X=df_avg)